### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [30]:
## OpenAi api key and open sourse models (llama3 , gemma2 , mistral --groq)
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [31]:


os.environ["OPENAI_API_KEY"]= os.getenv("OPEN_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
## langsmith Tracking 
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]= "true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["OPENAI_MODEL_NAME"]="openai/gpt-oss-20b:free"
groq_api_key= os.getenv("GROQ_API_KEY")  

In [32]:
from langchain_openai import ChatOpenAI 
from langchain_groq import ChatGroq

model = ChatGroq(model = "openai/gpt-oss-20b",groq_api_key = groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000015D8E779900>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015D8E779C90>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [38]:
from langchain_core.messages import HumanMessage,SystemMessage

message = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?") 
]

# call the model and display the response
result = model.invoke(message).content
print(result)


Bonjour, comment ça va?


In [39]:
 ## with parser 
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
result_with_parser = parser.invoke(result)
result_with_parser

'Bonjour, comment ça va?'

In [42]:
### using LCEL chain the components 

chain = model|parser
chain.invoke(message)

'Bonjour, comment allez‑vous ?'

In [43]:
### Prompt templates with LCEL

from langchain_core.prompts import ChatPromptTemplate

generic_template = "translate the following into {language}:"

prompt = ChatPromptTemplate(
    [("system",generic_template),("user","{text}")]
)

In [47]:
result= prompt.invoke({"language":"French","text":"Hello"})

In [48]:
result.to_messages()

[SystemMessage(content='translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [49]:
chain = prompt|model|parser 

chain.invoke({"language":"Arabic","text":"Hello"})

'مرحباً'